In [1]:
import urllib.request as req
from bs4 import BeautifulSoup
import os, os.path, gzip
import struct 

def decoding_mnist_rawData(dataStyle='train',maxCount=0):
    label_f=open(f'./data/mnist/{dataStyle}-labels-idx1-ubyte','rb')
    image_f=open(f'./data/mnist/{dataStyle}-images-idx3-ubyte','rb')
    csv_f=open(f'./data/mnist/{dataStyle}.csv','w',encoding='utf-8')
    magic_number, label_count = struct.unpack('>II',label_f.read(8))
    magic_number2, image_count, row, col = struct.unpack('>IIII',image_f.read(16))

    if maxCount>image_count:
        print('개수의 범위를 넘었습니다. 최소 %s개 이내' % image_count)
        return
    elif maxCount== -1:
        maxCount = image_count
    elif maxCount <-1:
        print('개수의 범위를 넘었습니다. 최소 %s개 이내' % image_count)
        return
    
    pixels=row*col 
    for idx in (range(maxCount)):
#         if idx >= maxCount : break
        label_tmp = struct.unpack('B', label_f.read(1))
        label=label_tmp[0]
        binarryData = image_f.read(pixels)
        strData = list(map(lambda x:str(x), binarryData))
        csv_f.write(str(label)+',')
        csv_f.write(','.join(strData) + '\n')

    image_f.close()
    label_f.close()
    csv_f.close()
    


In [2]:
decoding_mnist_rawData(dataStyle='train',maxCount=30000)
decoding_mnist_rawData(dataStyle='t10k',maxCount=10000)

In [3]:
def load_csv_ex(dataType='train'):
    labels=list()
    images=list()
    with open(f'./data/mnist/{dataType}.csv','r') as f:
        for line in f:
            tmp=line.split(',')
            labels.append(int(tmp[0]))
            images.append(list(map(lambda x:int(x)/,tmp[1:])))
    return {'labels':labels,'images':images}

In [12]:
train=load_csv_ex()
test=load_csv_ex('t10k')

In [16]:
train['labels'][:5],train['images'][0][:5]

([5, 0, 4, 1, 9], [0, 0, 0, 0, 0])

In [5]:
from sklearn import svm, model_selection, metrics

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_std_train=scaler.fit_transform(train['images'])

In [18]:
clf=svm.SVC()

In [19]:
len(train['images']),len(train['labels'])

(30000, 30000)

In [20]:
clf.fit(X_std_train, train['labels'])

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [21]:
predict=clf.predict(scaler.transform(test['images']))

In [22]:
metrics.accuracy_score(test['labels'],predict)

0.9582

In [23]:
clf_report=metrics.classification_report(test['labels'],predict)
print(clf_report)

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       980
           1       0.98      0.99      0.99      1135
           2       0.95      0.96      0.95      1032
           3       0.95      0.96      0.96      1010
           4       0.96      0.96      0.96       982
           5       0.96      0.94      0.95       892
           6       0.97      0.97      0.97       958
           7       0.92      0.95      0.94      1028
           8       0.95      0.94      0.95       974
           9       0.97      0.93      0.95      1009

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000

